In [191]:
import numpy as np
import matplotlib.pyplot as plt
from math import e

In [192]:
def defsize(d,m,k,n_class):
    return d,m,k,n_class

In [193]:
def randomize(D,M,K):
    W1 = np.random.rand(D,M)
    b1 = np.random.rand(M)
    W2 = np.random.rand(M,K)
    b2 = np.random.rand(K)
    
    return W1,b1,W2,b2

In [194]:
def datanize(D,K,N):
    X1 = np.random.rand(N,D) + np.array([3,0])
    X2 = np.random.rand(N,D) + np.array([0,3])
    X3 = np.random.rand(N,D) + np.array([-3,-3])
    X = np.vstack((X1,X2,X3))
    
    T_vector = np.array([0]*N+[1]*N + [2] * N)
    
    T_matrix = np.zeros((int(N*K),K))
    for i in range(K):
        T_matrix[500*int(i-1):500*int(i),i] = 1 
    
    return X,T_vector, T_matrix

In [195]:
def sigmoid(a):
    return 1/(1+np.exp(-a))

In [196]:
def softmax(a):
    return np.exp(a)/np.sum(np.exp(a),keepdims=True)

In [197]:
def forward(X,W1,b1,W2,b2):
    Z_pre = np.dot(X,W1)+ b1
    Z = sigmoid(Z_pre)
    a = np.dot(Z,W2) + b2
    Y = softmax(a)
    
    return Z,Y 

In [198]:
def cost(T_matrix,Y):
    return np.sum(np.multiply(T_matrix,Y))

In [213]:
def classification_rate(T_vector,Y):
    Y_vector = np.argmax(Y,axis=1)
    total = Y_vector.shape[0]
    print(Y_vector[:10])
    print(T_vector[:10])
    correct = np.sum(Y_vector == T_vector)
    
    return correct/total

4 ways to implement the derivation. Use deri_w2 as example(top to bot, slow to fast)

In [219]:
#slowest
def deri_w2(T_matrix,Y,Z):
    #W2 = np.random.rand(M,K)
    N,M = Z.shape
    K = T_matrix.shape[1]
    deri_w2 = np.zeros((M,K))
    for n in range(N):
        for m in range(M):
            for k in range(K):
                deri_w2[m,k] += (T_matrix[n,k] - Y[n,k])*Z[n,m]
    
    return deri_w2

In [224]:
def deri_w2_1(T_matrix,Y,Z):
    N,M = Z.shape
    K = T_matrix.shape[1]
    deri_w2 = np.zeros((M,K))
    for n in range(N):
        for k in range(K):
            #T-Y is constant, Z[n,:] and deri[:,k] are M length vector
            deri_w2[:,k] += (T_matrix[n,k] - Y[n,k])*Z[n,:] 
    
    return deri_w2

In [225]:
def deri_w2_2(T_matrix,Y,Z):
    N,M = Z.shape
    K = T_matrix.shape[1]
    deri_w2 = np.zeros((M,K))
    
    for n in range(N):
        deri_w2 += np.outer(Z[n,:],T_matrix[n,:]-Y[n,:]) #M*K
    return deri_w2

In [ ]:
#fastest
def deri_w2_3(T_matrix,Y,Z):
    N,M = Z.shape
    K = T_matrix.shape[1]
    deri_w2 = Z.T.dot(T-Y) #knowledge of linear algebra
    
    return deri_w2

In [201]:
def deri_b2(T_matrix,Y):
    return np.sum(T_matrix-Y, axis = 0)
# b2 is the specification of w2, here  z(n,m) = 1 always, so it can be transferred from deri_w2

In [202]:
def deri_w1(T_matrix,Y,W2,Z,X):
    N,M = Z.shape
    K = T_matrix.shape[1]
    D = X.shape[1]
    deri_w1 = np.zeros((D,M))
    
    for n in range(N):
        for m in range(M):
            for d in range(D):
                for k in range(K): #Z[1500,4=M]
                    deri_w1[d,m] += (T_matrix[n,k] - Y[n,k])*W2[m,k]*Z[n,m]*(1-Z[n,m])*X[n,d]
                    
    return deri_w1

In [203]:
def deri_b1(T_matrix,Y,W2,Z):
    N,M = Z.shape
    K = T_matrix.shape[1]
    deri_b1 = np.zeros(M)
    
    for n in range(N):
        for m in range(M):
            for k in range(K): #Z[1500,4=M]
                deri_b1[m]+= (T_matrix[n,k] - Y[n,k])*W2[m,k]*Z[n,m]*(1-Z[n,m])*1
                    
    return deri_b1

In [217]:
def ann(eta,epi):
    D,M,K,N = defsize(d=2, m=4, k=3, n_class=500)
    W1,b1,W2,b2 = randomize(D,M,K)
    X, T_vector, T_matrix = datanize(D,K,N)
    costs = []
    for i in range(epi):
        Z, Y = forward(X,W1,b1,W2,b2)
        W2 += eta * deri_w2(T_matrix,Y,Z) #W2 <- W2 - eta * d(-log)/dw = W2 - eta* (-1) * deri_w2 #thats why here is +=
        W1 += eta * deri_w1(T_matrix,Y,W2,Z,X)
        b2 += eta * deri_b2(T_matrix,Y)
        b1 += eta * deri_b1(T_matrix,Y,W2,Z)
        if i % 100 == 0:
            c = cost(T_matrix,Y)
            r = classification_rate(T_vector,Y)
            costs.append(c)
            print("The cost is : "+str(c)+" . The classification rate is : "+str(r)+" .")

In [218]:
ann(1e-4,500)

[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
The cost is : 0.3265428804999796 . The classification rate is : 0.3333333333333333 .
[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
The cost is : 0.5030679473418662 . The classification rate is : 0.3333333333333333 .
[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
The cost is : 0.5008099421421485 . The classification rate is : 0.6446666666666667 .
[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
The cost is : 0.5003610855164803 . The classification rate is : 0.6666666666666666 .
[1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 0 0 0]
The cost is : 0.5001991955365086 . The classification rate is : 0.6666666666666666 .
